In [2]:
%pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 2.3 MB/s eta 0:00:17
   ---------------------------------------- 0.1/38.8 MB 1.7 MB/s eta 0:00:23
   ---------------------------------------- 0.2/38.8 MB 1.5 MB/s eta 0:00:26
   ---------------------------------------- 0.3/38.8 MB 1.7 MB/s eta 0:00:23
   ---------------------------------------- 0.4/38.8 MB 1.8 MB/s eta 0:00:22
    --------------------------------------- 0.5/38.8 MB 1.8 MB/s eta 0:00:21
    --------------------------------------- 0.6/38.8 MB 2.0 MB/s eta 0:00:20
    --------------------------------------- 0.8/38.8 MB 2.1 MB/s eta 0:00:18
    --------------------------------------- 0.9/38.8 MB 2.2 MB/s eta 0:00:18
   - -------------------------------------- 1.1/38.8 MB 2.3 MB/s eta 0:00:17
   - -------------------------------------- 1.2/38.8 MB 2.4 MB/s eta 0:00:16
   - -------------------------------------- 1.3/38.8 MB 2.4 MB/s eta 0:00:16
   - -


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np

def detect_edges(image):
    """
    Detect edges using the Marr-Hildreth (Laplacian of Gaussian) operator.
    :param image: Grayscale image as a NumPy array.
    :return: Binary edge map as a NumPy array.
    """
    # Step 1: Apply Gaussian blur
    smoothed_image = cv2.GaussianBlur(image, (5, 5), 2)

    # Step 2: Compute Laplacian of the image
    laplacian = cv2.Laplacian(smoothed_image, cv2.CV_64F)

    # Step 3: Detect zero-crossings (binary edge map)
    edge_image = np.zeros_like(laplacian)
    edge_image[np.abs(laplacian) > 0.05] = 255  # Threshold for zero-crossings

    return edge_image.astype(np.uint8)

In [2]:
import zlib

def encode_edges(edge_image):
    """
    Compress the binary edge image using zlib to simulate JBIG encoding.
    :param edge_image: Binary edge map as a NumPy array.
    :return: Compressed binary data.
    """
    # Flatten and compress the edge image
    compressed_data = zlib.compress(edge_image.tobytes())
    return compressed_data

In [3]:
def quantize_and_subsample(image, edge_image, q=4, d=5):
    """
    Quantize and subsample pixel values adjacent to edges.
    :param image: Grayscale image as a NumPy array.
    :param edge_image: Binary edge map as a NumPy array.
    :param q: Quantization level (bits).
    :param d: Subsampling step.
    :return: Quantized values and positions.
    """
    adj_pixels = []
    indices = np.argwhere(edge_image > 0)

    for i, j in indices:
        neighbors = image[max(0, i-1):i+2, max(0, j-1):j+2].flatten()
        adj_pixels.extend(neighbors)

    # Quantize adjacent pixel values
    quantized_values = (np.array(adj_pixels) // (256 // (2**q))) * (256 // (2**q))

    # Subsample pixel values
    subsampled_values = quantized_values[::d]
    return subsampled_values

In [4]:
import heapq
from collections import Counter

def huffman_encode(data):
    """
    Perform Huffman encoding on the given data.
    :param data: Iterable data to encode.
    :return: Huffman tree and encoded binary string.
    """
    freq = Counter(data)
    heap = [[weight, [symbol, ""]] for symbol, weight in freq.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])

    huffman_dict = dict(heapq.heappop(heap)[1:])
    encoded_data = "".join(huffman_dict[symbol] for symbol in data)
    return huffman_dict, encoded_data

In [5]:
from scipy.ndimage import laplace

def reconstruct_image(edge_image, quantized_values, image_shape):
    """
    Reconstruct the image by solving Laplace's equation for missing regions.
    :param edge_image: Binary edge map as a NumPy array.
    :param quantized_values: Quantized values adjacent to edges.
    :param image_shape: Shape of the original image.
    :return: Reconstructed image as a NumPy array.
    """
    reconstructed = np.zeros(image_shape, dtype=np.float64)

    # Fill known values (edges) into the reconstructed image
    reconstructed[edge_image > 0] = quantized_values[:np.sum(edge_image > 0)]

    # Solve Laplace's equation for missing values
    mask = reconstructed == 0
    while np.any(mask):
        laplace_values = laplace(reconstructed)
        reconstructed[mask] += laplace_values[mask]

    return reconstructed

In [6]:
import os
from PIL import Image

def process_images(image_folder, output_folder, q=4, d=5):
    """
    Main function to process images for edge-based compression.
    :param image_folder: Folder containing input images.
    :param output_folder: Folder to save outputs.
    :param q: Quantization level.
    :param d: Subsampling step.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_name in os.listdir(image_folder):
        if image_name.endswith(('.png', '.bmp')):
            image_path = os.path.join(image_folder, image_name)
            image = np.array(Image.open(image_path).convert('L'))

            # Step 1: Detect edges
            edge_image = detect_edges(image)

            # Step 2: Encode edges
            encoded_edges = encode_edges(edge_image)

            # Step 3: Quantize and subsample
            quantized_values = quantize_and_subsample(image, edge_image, q, d)

            # Step 4: Compress quantized data
            huffman_dict, compressed_data = huffman_encode(quantized_values)

            # Step 5: Reconstruct the image
            reconstructed_image = reconstruct_image(edge_image, quantized_values, image.shape)

            # Save results
            Image.fromarray(edge_image).save(os.path.join(output_folder, f"edges_{image_name}"))
            Image.fromarray(reconstructed_image.astype(np.uint8)).save(os.path.join(output_folder, f"reconstructed_{image_name}"))

            print(f"Processed {image_name}: Compression achieved.")

In [7]:
if __name__ == "__main__":
    # Parameters
    input_folder = "input_images"  # Folder containing input images (e.g., BMP/PNG)
    output_folder = "output_images"  # Folder to save output (edges and reconstructed images)
    quantization_level = 4  # Number of bits for quantization (e.g., 4 bits)
    subsampling_step = 5  # Subsampling step for adjacent pixel values

    # Ensure folders exist
    if not os.path.exists(input_folder):
        raise ValueError(f"Input folder '{input_folder}' does not exist. Add images to this folder.")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Call the main processing function
    process_images(input_folder, output_folder, quantization_level, subsampling_step)
    
    print("Image compression and reconstruction completed.")

c:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\ndimage\_filters.py:555: RuntimeWarning: overflow encountered in add
  output += tmp
C:\Users\harsh\AppData\Local\Temp\ipykernel_7688\2995669374.py:20: RuntimeWarning: invalid value encountered in add
  reconstructed[mask] += laplace_values[mask]
c:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\ndimage\_filters.py:555: RuntimeWarning: invalid value encountered in add
  output += tmp


KeyboardInterrupt: 